<a href="https://colab.research.google.com/github/Trisha-Annambhotla/Health-Data/blob/main/HealthData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')

In [ ]:
df.head()

,S.No,Study ID,STREET,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE
0,1,5-2005-1,S58 T N H B PHASE I,SATHUVACHARY,NaN,NaN,NaN,VELLORE,TAMIL NADU
1,2,5-2005-2,12/35 IIN ST,THOTTAPALAYAM,THOTTAPALAYAM,NaN,NaN,VELLORE,TAMIL NADU
2,3,5-2005-3,"33, ARUMUGAM STREET,","ARNAI, T.V.MALAI,",NaN,NaN,NaN,NaN,TAMIL NADU
3,4,5-2005-4,12/13 MANAGKATTUR,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,NaN,ERODE,TAMIL NADU
4,5,5-2005-5,18/9 ELANGOVADIGAL ST,VELLORE -9,NaN,NaN,NaN,NaN,TAMIL NADU


In [ ]:
df.describe()

,S.No,PINCODE
count,32144.000000,20376.000000
mean,16072.500000,606867.528563
std,9279.317863,103060.896204
min,1.000000,0.000000
25%,8036.750000,517503.000000
50%,16072.500000,632006.000000
75%,24108.250000,632602.000000
max,32144.000000,997659.000000


In [ ]:
df.isnull().sum()

,0
S.No,0
Study ID,0
STREET,2193
TOWNVILLAGE,1092
POST,25841
PINCODE,11768
TALUK,12001
DISTRICT,734
STATE,145


In [ ]:
nan_state_df = df[df['STATE'].isna()]

In [ ]:
pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
import numpy as np

# Official list of Indian states (proper case)
indian_states = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa",
    "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala",
    "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland",
    "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura",
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Delhi"
]

# Lowercase mapping for case-insensitive match
indian_states = [state.upper() for state in indian_states]

unmatched_count=0
unmatched_indices = []


def correct_state_name(state_name, idx):
    global unmatched_count
    if pd.isna(state_name):
        return state_name
    state_name = state_name.upper()
    match, score, match_index = process.extractOne(
        state_name, indian_states, scorer=fuzz.token_sort_ratio
    )
    if score > 80:
        return indian_states[match_index]
    else:
        unmatched_count += 1
        unmatched_indices.append(idx)
        return state_name

# Apply correction with index tracking
df['STATE'] = [correct_state_name(state, idx) for idx, state in df['STATE'].items()]

# Create DataFrame with unmatched entries
unmatched_df = df.loc[unmatched_indices].reset_index(drop=True)


print("Unmatched count: ",unmatched_count)


Unmatched count:  562


In [ ]:
unmatched_df.head()

,S.No,Study ID,STREET,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE
0,19,8-2005-19,"MANI BITAN,TULSIPUR POST",HINDOLKOTHI CUTTACK 753 010,NaN,NaN,NaN,CUTTACK,ORISSA
1,197,2-2006-52,"24-117,SAI NAGAR COLONY",CHITTOOR,NaN,417001.0,NaN,CHITTOOR,ANDHRA
2,217,3-2006-72,"4/1328 OPP. ZPH SCHOOL,","YERRAGUNTA, CUDDAPAH",NaN,NaN,NaN,CUDDAPAH,A.P
3,248,5-2006-103,"637, VAZHUTHUVAR ROAD",DHAARMAPURI,METTU PALAYAM,NaN,NaN,PONDICHERRY,PONDICHERRY
4,267,6-2006-122,13-169 BASAVANNA KATTA ST.,PALMANER 517 408,NaN,NaN,NaN,CHITTOOR,A.P.


In [ ]:
df['STATE'] = df['STATE'].str.replace(r'[.]', '', regex=True)

In [ ]:
df['STATE'] = df['STATE'].apply(lambda x: 'ANDHRA PRADESH' if isinstance(x, str) and x.strip().upper() in ['ANDHRA', 'AP','A P','APRADESH','A PRADESH','ANDRA P','ANDH'] else x)
df['STATE'] = df['STATE'].apply(lambda x: 'UTTAR PRADESH' if isinstance(x, str) and x.strip().upper() == 'UP' else x)
df['STATE'] = df['STATE'].apply(lambda x: 'ODISHA' if isinstance(x, str) and x.strip().upper() == 'ORISSA' else x)
df['STATE'] = df['STATE'].apply(lambda x: 'TAMIL NADU' if isinstance(x, str) and x.strip().upper() in ['TAMILNADU', 'TN','VELLORE','CHENNAI','TVMALAI','TVMALI','THIRUVANNAMALAI'] else x)
df['STATE'] = df['STATE'].apply(lambda x: 'PONDICHERRY' if isinstance(x, str) and x.strip().upper() in ['PUDUCHERRY','PUDUCERY'] else x)
df['STATE'] = df['STATE'].apply(lambda x: 'WEST BENGAL' if isinstance(x, str) and x.strip().upper() in ['WBENGAL','WEST','WESTBENGAL'] else x)


In [ ]:
import re
df['STATE'] = df['STATE'].apply(lambda x: "" if isinstance(x, str) and re.search(r'\d', x) else x)


In [ ]:
unique_names = [
    'NEPAL', 'HOLLAND', 'C.G', 'BHUTAN', 'CALIFORNIA', 'DAR-ES-SALAAM',
    'SELECT STATE', 'ETHIOPIA', 'KOSHI', 'LIBERIA', 'SRI LANKA',
    'ENGLAND', 'AL ASIMAH', 'DHAKA', 'SATKHIRA SADAR', 'KHULNA',
    'CHINA', 'GERMAN', 'OYO', 'NIGERIA', 'ETHIOPIA,', 'PAIKAR DOUL',
    'TANZANIA', 'LAKSHMIPUR', 'CHUADANGA', 'DELDUAR', 'CHITTAGONG',
    'KHULNAS', 'BOGURA', 'INDONESIA', 'SYLHET','BANGLADESH','USA','CG','BANGLADEH'
]

# Drop rows where the 'state' column matches any name in unique_names
df = df[~df['STATE'].isin(unique_names)]

In [ ]:
import pandas as pd
from rapidfuzz import process, fuzz
import numpy as np

# Official list of Indian states (proper case)
indian_states = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa",
    "Gujarat", "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala",
    "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland",
    "Odisha", "Punjab", "Rajasthan", "Sikkim", "Tamil Nadu", "Telangana", "Tripura",
    "Uttar Pradesh", "Uttarakhand", "West Bengal", "Delhi","Andaman & Nicobar", "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi", "Jammu & Kashmir", "Ladakh", "Lakshadweep", "Pondicherry"
]

# Lowercase mapping for case-insensitive match
indian_states = [state.upper() for state in indian_states]

unmatched_count=0
unmatched_indices = []


def correct_state_name(state_name, idx):
    global unmatched_count
    if pd.isna(state_name):
        return state_name
    state_name = state_name.upper()
    match, score, match_index = process.extractOne(
        state_name, indian_states, scorer=fuzz.token_sort_ratio
    )
    if score > 80:
        return indian_states[match_index]
    else:
        unmatched_count += 1
        unmatched_indices.append(idx)
        return state_name

# Apply correction with index tracking
df.loc[:, 'STATE'] = [correct_state_name(state, idx) for idx, state in df['STATE'].items()]

# Create DataFrame with unmatched entries
unmatched_df = df.loc[unmatched_indices]


print("Unmatched count: ",unmatched_count)


Unmatched count:  15


In [ ]:
unmatched_df['STATE']

,STATE
293,
1174,
2300,KER
10223,KADAPA
13533,
17728,MUMBAI
20091,THRISSUR
25048,DAMAN AND DIU
25693,NEW DELHI
26273,TIRUPPATHUR


In [ ]:
import requests

client_id = '96dHZVzsAutKvwuMV5twTOoKhfXoVb0WroJi3_JiGAwwgrcZwGqZ9ossBRxJp_wldDr8U60FF7APM6DEWOuG5Q=='
client_secret = 'lrFxI-iSEg_NC3Jw5Jahng-ltISoGVYfCaeHR3wVM7XfzKR65zGY8eEvP6gkjrcBGfGRVBa6-NY9mWBoFZwIcbaLWDIOWuc0'

token_url = 'https://outpost.mapmyindia.com/api/security/oauth/token'
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_response = requests.post(token_url, data=payload)
token = token_response.json().get('access_token')

print("Access Token:", token)


Access Token: 72d88580-be6f-4d23-a1f4-092cf0f63fcf


In [ ]:
no_state = df[df['STATE'].isna()]
no_state.describe()

,S.No,PINCODE
count,145.000000,16.000000
mean,8647.972414,345185.812500
std,9095.083071,314948.348306
min,25.000000,0.000000
25%,2502.000000,4268.750000
50%,4594.000000,517132.000000
75%,10089.000000,632003.000000
max,31300.000000,721305.000000


In [ ]:
new_df = df.drop(['Study ID','STREET'],axis=1)

In [ ]:
new_df.reset_index(drop=True, inplace=True)
new_df.index = new_df.index + 1


In [ ]:
new_df.head()

,S.No,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE
1,1,SATHUVACHARY,NaN,NaN,NaN,VELLORE,TAMIL NADU
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,NaN,VELLORE,TAMIL NADU
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,NaN,TAMIL NADU
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,NaN,ERODE,TAMIL NADU
5,5,VELLORE -9,NaN,NaN,NaN,NaN,TAMIL NADU


In [ ]:
new_df.describe()

,S.No,PINCODE
count,31763.000000,20241.000000
mean,16013.758146,610607.297120
std,9275.789439,91739.180739
min,1.000000,0.000000
25%,7973.500000,517551.000000
50%,16001.000000,632007.000000
75%,24031.500000,632602.000000
max,32144.000000,997659.000000


In [ ]:
new_df.to_csv('cleaned_dataset.csv', index=False)


In [ ]:
files.download('cleaned_dataset.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
new_df['PINCODE'] = df['PINCODE'].apply(lambda x: x if str(x).isdigit() and len(str(x)) == 6 else np.nan)


In [ ]:
new_df.head()

,S.No,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE
1,1,SATHUVACHARY,NaN,NaN,NaN,VELLORE,TAMIL NADU
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,NaN,VELLORE,TAMIL NADU
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,NaN,TAMIL NADU
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,NaN,ERODE,TAMIL NADU
5,5,VELLORE -9,NaN,NaN,NaN,NaN,TAMIL NADU


In [ ]:
new_df['DISTRICT'] = new_df['DISTRICT'].apply(lambda x: 'TIRUVANNAMALAI' if str(x).endswith('MALAI') else x)


In [ ]:
count_tiruvannamalai = new_df['DISTRICT'].value_counts().get('TIRUVANNAMALAI', 0)
count_tiruvannamalai


np.int64(2575)

In [ ]:
import pandas as pd
import numpy as np

# Make sure 'PINCODE' exists
if 'PINCODE' not in new_df.columns:
    new_df['PINCODE'] = ''

# Function to detect and move pincode from DISTRICT
def move_pincode(row):
    val = str(row['DISTRICT']).strip() if pd.notnull(row['DISTRICT']) else ''
    cleaned = val.replace(" ", "")

    if cleaned.isdigit() and len(cleaned) == 6:
        row['PINCODE'] = cleaned
        row['DISTRICT'] = np.nan
    return row

# Apply to each row
new_df = new_df.apply(move_pincode, axis=1)


In [ ]:
new_df.head()

,S.No,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE
1,1,SATHUVACHARY,NaN,NaN,NaN,VELLORE,TAMIL NADU
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,NaN,VELLORE,TAMIL NADU
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,NaN,TAMIL NADU
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,NaN,ERODE,TAMIL NADU
5,5,VELLORE -9,NaN,NaN,NaN,NaN,TAMIL NADU


In [ ]:
final_df = new_df.drop('PINCODE',axis=1)

In [ ]:
final_df.head()

,S.No,TOWNVILLAGE,POST,TALUK,DISTRICT,STATE
1,1,SATHUVACHARY,NaN,NaN,VELLORE,TAMIL NADU
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,VELLORE,TAMIL NADU
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,TAMIL NADU
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,ERODE,TAMIL NADU
5,5,VELLORE -9,NaN,NaN,NaN,TAMIL NADU


In [ ]:
# Address to search
address = "KANNAMANGALAM,TIRUVANNAMALAI,TAMIL NADU"

# MapmyIndia Geocoding API endpoint
geocode_url = f"https://atlas.mapmyindia.com/api/places/geocode"

# Set headers and query params
headers = {
    'Authorization': f'Bearer {token}'
}
params = {
    'address': address
}

# Send request
response = requests.get(geocode_url, headers=headers, params=params)
data = response.json()

# Extract PIN code
cop_result = data.get('copResults', {})

if isinstance(cop_result, dict) and 'pincode' in cop_result:
    postal_code = cop_result['pincode']
    print(f"Postal Code (PIN): {postal_code}")
else:
    print("Postal code not found or invalid response.")






Postal Code (PIN): 


In [ ]:
import requests

client_id = '96dHZVzsAutKvwuMV5twTOoKhfXoVb0WroJi3_JiGAwwgrcZwGqZ9ossBRxJp_wldDr8U60FF7APM6DEWOuG5Q=='
client_secret = 'lrFxI-iSEg_NC3Jw5Jahng-ltISoGVYfCaeHR3wVM7XfzKR65zGY8eEvP6gkjrcBGfGRVBa6-NY9mWBoFZwIcbaLWDIOWuc0'

token_url = 'https://outpost.mapmyindia.com/api/security/oauth/token'
payload = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret
}

token_response = requests.post(token_url, data=payload)
token = token_response.json().get('access_token')

print("Access Token:", token)


Access Token: 72d88580-be6f-4d23-a1f4-092cf0f63fcf


In [ ]:
import pandas as pd
import numpy as np
import requests

# Make sure you have your API token
token = "72d88580-be6f-4d23-a1f4-092cf0f63fcf"

# Columns to construct the address from
address_columns = ['TOWNVILLAGE', 'POST', 'TALUK', 'DISTRICT', 'STATE']

# Function to create address string and get pincode
def get_pincode_from_address(row):
    # Safely combine non-null, non-empty values into a single address
    parts = [str(row[col]).strip() for col in address_columns if pd.notna(row[col]) and str(row[col]).strip()]
    address = ",".join(parts)

    # Skip if address is empty
    if not address:
        return np.nan

    # API request
    geocode_url = "https://atlas.mapmyindia.com/api/places/geocode"
    headers = {
        'Authorization': f'Bearer {token}'
    }
    params = {
        'address': address
    }

    try:
        response = requests.get(geocode_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        cop_result = data.get('copResults', {})

        if isinstance(cop_result, dict) and 'pincode' in cop_result:
            return cop_result['pincode']
    except Exception as e:
        print(f"Error fetching pincode for address '{address}': {e}")

    return np.nan

# Apply function to first 30 rows
final_df.loc[:29, 'predicted_pincode'] = final_df.loc[:29].apply(get_pincode_from_address, axis=1)


In [ ]:
new_df.head(20)

,S.No,TOWNVILLAGE,POST,PINCODE,TALUK,DISTRICT,STATE,predicted_pincode
1,1,SATHUVACHARY,NaN,NaN,NaN,VELLORE,TAMIL NADU,
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,NaN,VELLORE,TAMIL NADU,632004
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,NaN,TAMIL NADU,614616
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,NaN,ERODE,TAMIL NADU,638316
5,5,VELLORE -9,NaN,NaN,NaN,NaN,TAMIL NADU,
6,6,"WALAJAH T.K, VELLORE",NaN,NaN,NaN,NaN,TAMIL NADU,632004
7,7,KOVILUR,KOVILUR VANDAVASI,NaN,NaN,TIRUVANNAMALAI,TAMIL NADU,635703
8,8,CHITTOOR,CHITTOOR,NaN,NaN,CHITTOOR,ANDHRA PRADESH,
9,9,"THIRUVANNAMALAI,",NaN,NaN,NaN,TIRUVANNAMALAI,TAMIL NADU,606601
10,10,DIGRI POST 721 252,NaN,NaN,NaN,MIDNAPURE W,WEST BENGAL,721253


In [ ]:
final_df.head(30)

,S.No,TOWNVILLAGE,POST,TALUK,DISTRICT,STATE,predicted_pincode
1,1,SATHUVACHARY,NaN,NaN,VELLORE,TAMIL NADU,
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,VELLORE,TAMIL NADU,632004
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,TAMIL NADU,614616
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,ERODE,TAMIL NADU,638316
5,5,VELLORE -9,NaN,NaN,NaN,TAMIL NADU,
6,6,"WALAJAH T.K, VELLORE",NaN,NaN,NaN,TAMIL NADU,632004
7,7,KOVILUR,KOVILUR VANDAVASI,NaN,TIRUVANNAMALAI,TAMIL NADU,635703
8,8,CHITTOOR,CHITTOOR,NaN,CHITTOOR,ANDHRA PRADESH,
9,9,"THIRUVANNAMALAI,",NaN,NaN,TIRUVANNAMALAI,TAMIL NADU,606601
10,10,DIGRI POST 721 252,NaN,NaN,MIDNAPURE W,WEST BENGAL,721253


In [ ]:
final_df.drop('predicted_pincode',axis=1)

,S.No,TOWNVILLAGE,POST,TALUK,DISTRICT,STATE
1,1,SATHUVACHARY,NaN,NaN,VELLORE,TAMIL NADU
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,VELLORE,TAMIL NADU
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,TAMIL NADU
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,ERODE,TAMIL NADU
5,5,VELLORE -9,NaN,NaN,NaN,TAMIL NADU
...,...,...,...,...,...,...
31759,32140,IRUVARAM CHECKPOST,NaN,CHITTOOR,CHITTOOR,ANDHRA PRADESH
31760,32141,VELLORE,NaN,NaN,VELLORE,TAMIL NADU
31761,32142,VELLORE,NaN,THORAPADI,VELLORE,TAMIL NADU
31762,32143,"NEW COLONY, THIRUNAGAR,",NaN,"KALINJUR, GANDHINAGAR",VELLORE,TAMIL NADU


In [ ]:
import pandas as pd
import requests
import json

# Sample API Key from data.gov.in (limited access)
api_key = "579b464db66ec23bdd000001cdd3946e44ce4aad7209ff7b23ac571b"
base_url = "https://api.data.gov.in/resource/5c2f62fe-5afa-4119-a499-fec9d604d5bd"

# Function to fetch PINCODE using supported filters
def fetch_pincode(row):
    # Extract only valid fields; fill NaNs with empty string
    officename = str(row.get('POST', '')).strip() if pd.notna(row.get('POST')) else ''
    district = str(row.get('DISTRICT', '')).strip() if pd.notna(row.get('DISTRICT')) else ''
    statename = str(row.get('STATE', '')).strip() if pd.notna(row.get('STATE')) else ''

    # Build the filters dictionary with valid field names
    filters = {
        "officename": officename,
        "district": district,
        "statename": statename
    }

    # Remove any blank filters
    filters = {f"filters[{k}]": v for k, v in filters.items() if v}

    # Add base params
    params = {
        "api-key": api_key,
        "format": "json",
        "limit": 1  # return only the first match
    }
    params.update(filters)

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("records"):
                return data["records"][0].get("pincode", "Not Found")
            else:
                return "Not Found"
        else:
            return f"Error {response.status_code}"
    except Exception as e:
        return "Error"

# Apply only to the first 10 rows
subset_df = final_df.head(10).copy()

# Fetch pincode using the API
subset_df["PINCODE"] = subset_df.apply(fetch_pincode, axis=1)

# Output the result
print(subset_df[["TOWNVILLAGE", "POST", "TALUK", "DISTRICT", "STATE", "PINCODE"]])


             TOWNVILLAGE                POST TALUK        DISTRICT  \
1           SATHUVACHARY                 NaN   NaN         VELLORE   
2          THOTTAPALAYAM       THOTTAPALAYAM   NaN         VELLORE   
3      ARNAI, T.V.MALAI,                 NaN   NaN             NaN   
4          KAVUNTHAMPADI        AYYAMPALAYAM   NaN           ERODE   
5             VELLORE -9                 NaN   NaN             NaN   
6   WALAJAH T.K, VELLORE                 NaN   NaN             NaN   
7                KOVILUR  KOVILUR  VANDAVASI   NaN  TIRUVANNAMALAI   
8               CHITTOOR            CHITTOOR   NaN        CHITTOOR   
9       THIRUVANNAMALAI,                 NaN   NaN  TIRUVANNAMALAI   
10    DIGRI POST 721 252                 NaN   NaN     MIDNAPURE W   

             STATE    PINCODE  
1       TAMIL NADU     632516  
2       TAMIL NADU  Not Found  
3       TAMIL NADU     632502  
4       TAMIL NADU     638455  
5       TAMIL NADU     632502  
6       TAMIL NADU     632502  
7    

In [ ]:
subset_df.head(10)

,S.No,TOWNVILLAGE,POST,TALUK,DISTRICT,STATE,predicted_pincode,PINCODE
1,1,SATHUVACHARY,NaN,NaN,VELLORE,TAMIL NADU,,632516
2,2,THOTTAPALAYAM,THOTTAPALAYAM,NaN,VELLORE,TAMIL NADU,632004,Not Found
3,3,"ARNAI, T.V.MALAI,",NaN,NaN,NaN,TAMIL NADU,614616,632502
4,4,KAVUNTHAMPADI,AYYAMPALAYAM,NaN,ERODE,TAMIL NADU,638316,638455
5,5,VELLORE -9,NaN,NaN,NaN,TAMIL NADU,,632502
6,6,"WALAJAH T.K, VELLORE",NaN,NaN,NaN,TAMIL NADU,632004,632502
7,7,KOVILUR,KOVILUR VANDAVASI,NaN,TIRUVANNAMALAI,TAMIL NADU,635703,604408
8,8,CHITTOOR,CHITTOOR,NaN,CHITTOOR,ANDHRA PRADESH,,517001
9,9,"THIRUVANNAMALAI,",NaN,NaN,TIRUVANNAMALAI,TAMIL NADU,606601,632511
10,10,DIGRI POST 721 252,NaN,NaN,MIDNAPURE W,WEST BENGAL,721253,Not Found


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load your dataset
your_df = pd.read_csv('your_dataset.csv')

# Load the large reference dataset
ref_df = pd.read_csv('reference_dataset.csv')

# Extract unique districts from reference
unique_ref_districts = ref_df['District'].dropna().unique().tolist()

print(f"Unique districts in reference: {len(unique_ref_districts)}")

# Load pretrained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Precompute embeddings for reference districts (do once)
ref_embeddings = model.encode(unique_ref_districts, convert_to_tensor=True)

def match_district(district_name):
    if pd.isna(district_name) or district_name.strip() == '':
        return ("", 0.0)  # handle empty values gracefully
    query_embedding = model.encode(district_name, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, ref_embeddings)[0]
    best_score, best_idx = cosine_scores.max(), cosine_scores.argmax()
    return unique_ref_districts[best_idx], best_score.item()

# Apply matching function to your dataset's DISTRICT column
your_df[['matched_district', 'similarity_score']] = your_df['DISTRICT'].apply(
    lambda x: pd.Series(match_district(x))
)

# Optional: filter out low-confidence matches
threshold = 0.7
your_df['final_match'] = your_df.apply(
    lambda row: row['matched_district'] if row['similarity_score'] > threshold else "No confident match",
    axis=1
)

# Save result to a new CSV
your_df.to_csv('your_dataset_matched.csv', index=False)

print("Matching completed!")
